# SAQ VIGIE - METHODOLOGY

### Loading Libraries and Modules

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Global Parameters
stop_words = set(stopwords.words('english'))

### Connecting to Twitter's API

In [ ]:
import tweepy
import webbrowser
import time

consumer_key = "Qt7XQIJW9Ey19v8w3QtSELMTb"
consumer_secret_key = "tXdjubhBT0QUrsTIaF9SzTptKxPE9TZohruePiGvlrydwDnebx"

callback_url = 'oob' 

# Authenticates the application
auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key, callback_url)

# Grab a URL that we can send our user to
redirect_url = auth.get_authorization_url()

# Test the Twitter API Application URL 
print(redirect_url)

In [ ]:
webbrowser.open(redirect_url)

In [ ]:
# Create a PIN form in order to authenticate the URL even if the PIN value changes
user_pin_input = input("What's the PIN value? ")

In [ ]:
user_pin_input

In [ ]:
# Access keys for the particular user. These don't change.
auth.get_access_token(user_pin_input)

In [ ]:
api = tweepy.API(auth)

In [ ]:
# Testing the API with my own Twitter Profile
me = api.me()

In [ ]:
# Printing my username
print(me.screen_name)

### Getting the tweets

In [ ]:
# SAQ

tweets = tweepy.Cursor(api.search,q="@LaSAQ_officiel",count=20000000,since="2022-06-01", wait_on_rate_limit=True).items()
all_tweets = [[tweet.text, tweet.created_at, tweet.text, tweet.user, tweet.user.location] for tweet in tweets]

SAQ_1 = pd.DataFrame(data=all_tweets, columns=['tweet','created at','text','user', "location"])
SAQ_1

In [ ]:
# SAQ

tweets = tweepy.Cursor(api.search,q="SAQ",count=20000000, since="2022-06-01",wait_on_rate_limit=True).items()
all_tweets = [[tweet.text, tweet.created_at, tweet.text, tweet.user, tweet.user.location] for tweet in tweets]

SAQ_2 = pd.DataFrame(data=all_tweets, columns=['tweet','created at','text','user', "location"])
SAQ_2


In [ ]:
# SOCIÉTÉ DES ALCOOLS DU QUÉBEC

tweets = tweepy.Cursor(api.search,q="Société des alcools du Québec",count=20000000, since="2022-06-01",wait_on_rate_limit=True).items()
all_tweets = [[tweet.text, tweet.created_at, tweet.text, tweet.user, tweet.user.location] for tweet in tweets]

SAQ_3 = pd.DataFrame(data=all_tweets, columns=['tweet','created at','text','user', "location"])
SAQ_3

### Cleaning the tweets

In [ ]:
# Concatenate the three different outputs from the API

frames = [SAQ_1, SAQ_2, SAQ_3]
  
result = pd.concat(frames)
result

In [ ]:
# Cleaning and separating hashtags, links, mentions

import re 

def remove_punct(text):
    text = re.sub(r"http\S+",' ', text)
    text = re.sub(r'\B@\w*[a-zA-Z]+\w*',' ', text)
    text = re.sub(r'RT','', text)
    text = re.sub(r':','', text)
    text = re.sub(r'\B#\w*[a-zA-Z]+\w*',' ', text)
    text = re.sub(r"@(\w+)", ' ',text, flags=re.MULTILINE)
    return text

result['tweet'] = result['tweet'].apply(str)
result['Tweet_Clean'] = result['tweet'].apply(lambda x: remove_punct(x))
result['Mentions'] = result['tweet'].apply(lambda x: re.findall(r'\B@\w*[a-zA-Z]+\w*', x))
result['hashtag'] = result['tweet'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', x))

result.head(10)

In [ ]:
# Language detector to separate EN/FR

langue = []

from langdetect import detect

for i in result["tweet"]:
    try:
        a = detect(i)
    except:
        language = "error"
        print("This row throws and error:", i[0])
    langue.append(a)

result["langue"]=langue  

In [ ]:
# Separating EN/FR

import pandas as pd

# DF FRANCAIS
fr = result[(result.langue == "fr")]

# DF ANGLAIS
en = result[(result.langue == "en")]

### Sentiment Analysis FR

In [ ]:
# ANALYSE SENTIMENT - FRANCAIS
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

senti_list = []
for i in fr["tweet"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')   

polarity = []
for i in fr["tweet"]:
    vs = tb(i).sentiment[0]
    polarity.append(vs)

fr["polarity"]=polarity    
fr["sentiment"]=senti_list
fr.head()

### Sentiment Analysis EN

In [ ]:
# ANALYSE SENTIMENT - ANGLAIS
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from textblob import Blobber
tb = Blobber()

senti_list_en = []
for i in en["tweet"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list_en.append('Positive')
    elif (vs < 0):
        senti_list_en.append('Negative')
    else:
        senti_list_en.append('Neutral')   

polarity_en = []
for i in en["tweet"]:
    vs = tb(i).sentiment[0]
    polarity_en.append(vs)

en["polarity"]=polarity_en   
en["sentiment"]=senti_list_en
en.head()

### Wordcloud

In [ ]:
# Python program to generate WordCloud
 
# importing all necessary modules
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
 
df = fb
 
comment_words = ''
stopwords = ["pendant","qu","michel  béland","patrick  ferland","jerome lagrange","michel béland","aux","patrice gravel","nan","nan nan","voir","plus","c","n","l","tony","maxime","d","m","aux,""vers","richard","mets","n'ai","andré","allons","qu'il","d'autres","faut","qu'ils","ont","s'ils","main","n'ont","ca","n'est","un","leurs","ne","une","c'est","s'est","C'est","telle","se","temp","va","y","je","pat77432407","à","j'ai","lasaq_officiel","de","rt","saq","https","co","alors","au","aucuns","aussi","autre","avant","avec","avoir","bon","car","ce","cela","ces","ceux","chaque","ci","comme","comment","dans","des","du","dedans","dehors","depuis","devrait","doit","donc","dos","début","elle","elles","en","encore","essai","est","et","eu","fait","faites","fois","font","hors","ici","il","ils","je	juste","la","le","les","leur","là","ma","maintenant","mais","mes","mien","moins","mon","mot","même","ni","nommés","notre","nous","ou","où","par","parce","pas","peut","peu","plupart","pour","pourquoi","quand","que","quel","quelle","quelles","quels","qui","sa","sans","ses","seulement","si","sien","son","sont","sous","soyez	sujet","sur","ta","tandis","tellement","tels","tes","ton","tous","tout","trop","très","tu","voient","vont","votre","vous","vu","ça","étaient","état","étions","été","être","i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"]
 
# iterate through the csv file
for val in df.Commentaire:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, 
                height = 800, 
                max_words = 30,
                background_color ='white',
                stopwords = stopwords,
                font_path='Arial',
                colormap='Reds').generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()